In [1]:
import functools
import inspect
import logging
import os
import random
import warnings
from typing import Any, Optional, Union

import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from src.qrt_stock_returns.utils import conf_params, get_node_output, run_pipeline_node

warnings.filterwarnings("ignore")

[02/09/25 18:06:40] INFO     Using 'conf/logging.yml' as logging configuration. You can change this ]8;id=871169;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=326950;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/framework/project/__init__.py#269\269]8;;\
                             by setting the KEDRO_LOGGING_CONFIG environment variable accordingly.                 

[02/09/25 18:06:41] WARNING  /Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-p ]8;id=483085;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=369024;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/warnings.py#112\112]8;;\
                             ackages/mlflow/pyfunc/utils/data_validation.py:134: UserWarning:                      
                             [33mAdd type hints to the `predict` method to enable data validation                  
                             and automatic signature inference during model logging. Check                         
                             https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage                
                             -in-pythonmodel for more details.[0m                                                  
                               color_warning(                                                                      
                                                                                                                   

                    INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=253596;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=193565;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#67\67]8;;\

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=636796;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro_mlflow/config/kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=13763;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro_mlflow/config/kedro_mlflow_config.py#260\260]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlflow_runs'). It                           
                             is converted to a valid uri:                                                          
                             'file:///Users/krzysztofwojdalski/github_projects/qrt-stock                           
                             -returns/mlflow_runs'                                                                 

                    WARNING  /Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/import ]8;id=244949;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=296512;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/warnings.py#112\112]8;;\
                             lib/__init__.py:90: KedroDeprecationWarning: `tracking.JSONDataset`                   
                             and `tracking.MetricsDataset` are deprecated. These datasets will be                  
                             removed in kedro-datasets 7.0.0                                                       
                               return _bootstrap._gcd_import(name[level:], package, level)                         
                                                                                                                   

/Users/krzysztofwojdalski/github_projects/qrt-stock-returns/src/qrt_stock_returns


[02/09/25 18:06:45] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=778597;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=899849;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

In [2]:

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
log.info("Starting XGBoost")

[02/09/25 18:06:46] INFO     Starting XGBoost                                                       ]8;id=815150;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/1993078283.py\1993078283.py]8;;\:]8;id=51432;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/1993078283.py#3\3]8;;\

### Loading Up Kedro Config
Grab all our parameters from the config file
* This has elements like our target variable and k-fold settings

In [3]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)


seed_everything()

In [4]:
class Config:
    DATA_DIR: str = "data"
    L1_N_TRIALS: int = 1
    L2_N_TRIALS: int = 100
    N_JOBS: int = 2
    OPTUNA_STORAGE: str = "sqlite:///db.sqlite3"

    MODELS: dict[str, str] = {
        # "lr": "Logistic Regression",
        # "ada": "AdaBoost",
        # "rf": "Random Forest",
        "xgb": "XGBoost",
        "lgb": "LightGBM",
    }

    @classmethod
    def filepath(cls, filename: str):
        return os.path.join(cls.DATA_DIR, filename)

In [5]:
class LRConfig:
    DEFAULT_VALUES: dict[str, Union[float, str]] = {
        "tol": 1e-4,
        "C": 1.0,
        "solver": "lbfgs",
    }
    STATIC_PARAMS: dict[str, Union[int, bool]] = {
        "max_iter": 1000,
        "verbose": False,
    }

    USE_PRUNER: bool = False

    @classmethod
    def get_fit_params(
        cls,
        X_train: pd.DataFrame,
        y_train: pd.Series,
        X_val: pd.DataFrame,
        y_val: pd.Series,
        params: dict[str, Any],
    ):
        return {}


class AdaConfig:
    DEFAULT_VALUES: dict[str, Union[None, int, float]] = {
        "estimator": None,
        "n_estimators": 50,
        "learning_rate": 1.0,
    }
    STATIC_PARAMS: dict[str, Any] = {}

    USE_PRUNER: bool = False

    @classmethod
    def get_fit_params(
        cls,
        X_train: pd.DataFrame,
        y_train: pd.Series,
        X_val: pd.DataFrame,
        y_val: pd.Series,
        params: dict[str, Any],
    ):
        return {}

In [6]:
class RFConfig:
    DEFAULT_VALUES: dict[str, Union[None, int, str, float, bool]] = {
        "n_estimators": 100,
        "max_depth": None,
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "max_features": "sqrt",
        "bootstrap": True,
        "ccp_alpha": 0.0,
        "max_samples": None,
    }

    STATIC_PARAMS: dict[str, int] = {
        "n_jobs": Config.N_JOBS,
    }

    USE_PRUNER: bool = False

    @classmethod
    def get_fit_params(
        cls,
        X_train: pd.DataFrame,
        y_train: pd.Series,
        X_val: pd.DataFrame,
        y_val: pd.Series,
        params: dict[str, Any],
    ):
        return {}

In [7]:
class XGBConfig:
    EVAL_METRIC: str = "logloss"

    DEFAULT_VALUES: dict[str, Union[int, float, str, None]] = {
        "max_depth": 6,
        "n_estimators": 100,
        "alpha": 0.0,
        "lambda": 1.0,
        "learning_rate": 0.3,
        "colsample_bytree": 1.0,
        "colsample_bylevel": 1.0,
        "min_child_weight": 1.0,
        "early_stopping_rounds": None,
    }

    STATIC_PARAMS: dict[str, Union[str, bool, int]] = {
        "tree_method": "hist",
        "use_label_encoder": False,
        "n_jobs": Config.N_JOBS,
        "predictor": "gpu_predictor",
        "max_bin": 1024,
        "eval_metric": EVAL_METRIC,
    }

    USE_PRUNER: bool = True

    @classmethod
    def get_fit_params(
        cls,
        X_train: pd.DataFrame,
        y_train: pd.Series,
        X_val: pd.DataFrame,
        y_val: pd.Series,
        params: dict[str, Any],
    ):
        return {
            "eval_set": [(X_train, y_train), (X_val, y_val)],
            "verbose": False,
        }

In [8]:
class LGBConfig:
    DEFAULT_VALUES: dict[str, Union[int, float]] = {
        "num_leaves": 31,
        "max_depth": -1,
        "learning_rate": 0.1,
        "n_estimators": 100,
        "reg_alpha": 0.0,
        "reg_lambda": 0.0,
        "min_child_samples": 20,
        "subsample_for_bin": 200000,
    }

    STATIC_PARAMS: dict[str, Union[int, str]] = {
        "n_jobs": Config.N_JOBS,
        "verbose": -1,
        "objective": "binary",
    }

    USE_PRUNER: bool = True

    @classmethod
    def get_fit_params(
        cls,
        X_train: pd.DataFrame,
        y_train: pd.Series,
        X_val: pd.DataFrame,
        y_val: pd.Series,
        params: dict[str, Any],
    ):
        callbacks = params.get("callbacks", []) + [lgb.log_evaluation(period=0)]
        return {
            "eval_set": [(X_train, y_train), (X_val, y_val)],
            "eval_metric": "logloss",
            "callbacks": callbacks,
        }

In [9]:
CONFIG_MAP: dict[str, Any] = {
    "lr": LRConfig,
    "ada": AdaConfig,
    "rf": RFConfig,
    "xgb": XGBConfig,
    "lgb": LGBConfig,
}


MODEL_MAP: dict[str, Any] = {
    "lr": LogisticRegression,
    "ada": AdaBoostClassifier,
    "rf": RandomForestClassifier,
    "xgb": xgb.XGBClassifier,
    "lgb": lgb.LGBMClassifier,
}

In [10]:
def train(  # noqa
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
    model: str,
    params: dict[str, Any],
    verbose: bool = True,
):
    klass = MODEL_MAP[model]
    config = CONFIG_MAP[model]

    params = {k: v for k, v in params.items() if k in config.DEFAULT_VALUES}
    params.update({k: v for k, v in config.DEFAULT_VALUES.items() if k not in params})
    params.update(config.STATIC_PARAMS)

    clf = klass(**params)

    fit_params = config.get_fit_params(
        X_train=X_train, y_train=y_train, X_val=X_test, y_val=y_test, params=params
    )

    clf.fit(
        X=X_train,
        y=y_train,
        **fit_params,
    )

    train_preds = clf.predict_proba(X_train)[:, 1]
    test_preds = clf.predict_proba(X_test)[:, 1]

    y_pred = clf.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)

    if verbose:
        log.info(f"Accuracy = {acc:.4f}")

    return train_preds, test_preds, acc

In [11]:
def lr_objective(
    trial: optuna.Trial,
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
):
    params = {
        "tol": trial.suggest_float("tol", 1e-6, 1e-4, log=True),
        "C": trial.suggest_float("C", 0.5, 2.0, log=True),
        "solver": trial.suggest_categorical(
            "solver", ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
        ),
    }

    _, _, acc = train(
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test,
        model="lr",
        params=params,
        verbose=False,
    )
    return acc

In [12]:
def adaboost_objective(
    trial: optuna.Trial,
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 0.1, 5.0, log=True),
    }

    tune_estimator = trial.suggest_categorical("tune_estimator", [True, False])

    if tune_estimator:
        max_depth = trial.suggest_int("max_depth", 1, 50)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10, log=True)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5, log=True)
        ccp_alpha = trial.suggest_float("ccp_alpha", 0.01, 1.0, log=True)

        params["estimator"] = DecisionTreeClassifier(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            ccp_alpha=ccp_alpha,
        )

    _, _, acc = train(
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test,
        model="ada",
        params=params,
        verbose=False,
    )
    return acc

In [13]:
def rf_objective(
    trial: optuna.Trial,
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 1, 500),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "min_samples_split": trial.suggest_int("min_samples_plit", 2, 10, log=True),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5, log=True),
        "max_features": trial.suggest_categorical(
            "max_features", ["sqrt", "log2", None]
        ),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "ccp_alpha": trial.suggest_float("ccp_alpha", 0.01, 1.0, log=True),
    }

    if params["bootstrap"] is True:
        params["max_samples"] = trial.suggest_float("max_samples", 0.01, 1.0, log=True)

    _, _, acc = train(
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test,
        model="rf",
        params=params,
        verbose=False,
    )
    return acc

In [14]:
def xgb_objective(
    trial: optuna.Trial,
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "n_jobs": 1,
        "verbosity": 2,
        "max_depth": trial.suggest_int("max_depth", 2, 8),
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        # "learning_rate": 0.3,
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "gamma": trial.suggest_float("gamma", 0, 0.2),
        "subsample": trial.suggest_float("subsample", 0.8, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.8, 1.0),
        "max_delta_step": trial.suggest_int("max_delta_step", 0, 2),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 0.5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 0.5),
        "random_state": 42,
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.2, 1.0),
        "early_stopping_rounds": trial.suggest_int(
            "early_stopping_rounds", 5, 20, step=5
        ),
    }

    obs_k = f"validation_1-{XGBConfig.EVAL_METRIC}"
    params["callbacks"] = [optuna.integration.XGBoostPruningCallback(trial, obs_k)]

    _, _, acc = train(
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test,
        model="xgb",
        params=params,
        verbose=False,
    )
    return acc

In [15]:
def lgb_objective(
    trial: optuna.Trial,
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 31, 100, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 100, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 50, log=True),
        "subsample_for_bin": trial.suggest_int("subsample_for_bin", 2000, 8000),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 1.0),
    }

    params["callbacks"] = [
        optuna.integration.LightGBMPruningCallback(trial, "logloss", "valid_1")
    ]

    _, _, acc = train(
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test,
        model="lgb",
        params=params,
        verbose=False,
    )
    return acc


OBJECTIVE_MAP: dict[str, Any] = {
    "lr": lr_objective,
    "ada": adaboost_objective,
    "rf": rf_objective,
    "xgb": xgb_objective,
    "lgb": lgb_objective,
}

In [16]:
def hyperparameter_search(  # noqa
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
    model: str,
    n_trials: int = Config.L1_N_TRIALS,
):
    # Check if function is being called from parent frame
    frame = inspect.currentframe()
    if frame is not None:
        caller = frame.f_back
        if caller is not None:
            caller_name = caller.f_code.co_name
            log.info(f"Caller name: {caller_name}")
            if caller_name not in ["fit_level_one_models", "fit_level_two_model"]:
                raise RuntimeError(
                    "This function should only be called from fit_level_one_models or fit_level_two_model"
                )
            if caller_name == "fit_level_two_model":
                study_name = self.level_two_study_name
            else:
                study_name = f"ensemble_{model}"

    v = optuna.logging.get_verbosity()
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    objective = OBJECTIVE_MAP[model]
    objective = functools.partial(
        objective, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test
    )

    pruner = (
        optuna.pruners.HyperbandPruner()
        if CONFIG_MAP[model].USE_PRUNER is True
        else None
    )

    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(
        storage=Config.OPTUNA_STORAGE,  # Specify the storage URL here.
        study_name=study_name,
        direction="maximize",
        load_if_exists=True,
        pruner=pruner,
        sampler=sampler,
    )

    study.optimize(objective, n_trials=n_trials)

    optuna.logging.set_verbosity(v)

    return study.best_params

In [17]:
class Ensemble:
    def __init__(
        self,
        X_train: pd.DataFrame,
        X_test: pd.DataFrame,
        y_train: pd.Series,
        y_test: pd.Series,
        exclude: Optional[set] = None,
    ):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.level_two_study_name = (
            f"ensemble_lr_l2_{'_'.join(Config.MODELS.keys())}"  # for now only lr
        )
        self.level_one_study_names = {
            model: f"ensemble_l1_{model}" for model in Config.MODELS.keys()
        }

        models = Config.MODELS.keys()

        if exclude is not None:
            models = models - exclude

        self.models: list[str] = list(models)

        columns = [f"{m}_preds" for m in self.models]

        self.meta_train_df = pd.DataFrame(index=X_train.index)
        self.meta_test_df = pd.DataFrame(index=X_test.index)

        # Add storage for trained models
        self.level_one_models = {}
        self.level_two_model = None
        self.level_one_params = {}
        self.level_two_params = None

    def fit_level_one_models(self, search: bool = True):
        """Train level 1 models of the ensemble.

        For each model in self.models, either search for optimal hyperparameters using Optuna
        or load existing best parameters, then train the model and store predictions in
        meta_train_df and meta_test_df.

        Args:
                search: If True, perform hyperparameter search. If False, load existing best parameters.
        """
        log.info("Training level 1 models...")

        for model in self.models:
            log.info(f"{Config.MODELS[model]}:")

            if search:
                log.info("\tFinding optimal hyperparameters using Optuna...")
                params = hyperparameter_search(
                    X_train=self.X_train,
                    X_test=self.X_test,
                    y_train=self.y_train,
                    y_test=self.y_test,
                    model=model,
                )
                log.info(f"\n\tBest params: {params}\n")
            else:
                study = optuna.load_study(
                    storage=Config.OPTUNA_STORAGE,
                    study_name=self.level_one_study_names[model],
                )
                params = study.best_params
                log.info("\tUsing existing best parameters...\n")

            log.info("\tTraining model with optimal parameters...\n")

            # Store params
            self.level_one_params[model] = params

            # Train and store the model
            klass = MODEL_MAP[model]
            config = CONFIG_MAP[model]

            params.update(config.STATIC_PARAMS)
            clf = klass(**params)

            fit_params = config.get_fit_params(
                X_train=self.X_train,
                y_train=self.y_train,
                X_val=self.X_test,
                y_val=self.y_test,
                params=params,
            )

            clf.fit(X=self.X_train, y=self.y_train, **fit_params)
            self.level_one_models[model] = clf

            # Get predictions for meta features
            train_preds = clf.predict_proba(self.X_train)[:, 1]
            test_preds = clf.predict_proba(self.X_test)[:, 1]

            self.meta_train_df[f"{model}_preds"] = train_preds
            self.meta_test_df[f"{model}_preds"] = test_preds

    def fit_level_two_model(self, search: bool = True):
        """Train a Logistic Regression model as level 2 model.

        If search is True, perform hyperparameter search using Optuna.
        If search is False, load existing best parameters.
        """
        log.info("Training a Logistic Regression model as level 2 model...")

        if search:
            log.info("\tFinding optimal hyperparameters using Optuna...")
            params = hyperparameter_search(
                X_train=self.meta_train_df,
                X_test=self.meta_test_df,
                y_train=self.y_train,
                y_test=self.y_test,
                model="lr",
                n_trials=Config.L2_N_TRIALS,
            )
            log.info(f"\n\tBest params: {params}\n")
        else:
            study = optuna.load_study(
                storage=Config.OPTUNA_STORAGE,
                study_name=self.level_two_study_name,
            )
            params = study.best_params
            log.info("\tUsing existing best parameters...\n")

        self.level_two_params = params

        # Train and store the level 2 model
        klass = MODEL_MAP["lr"]
        config = CONFIG_MAP["lr"]

        params.update(config.STATIC_PARAMS)
        clf = klass(**params)

        fit_params = config.get_fit_params(
            X_train=self.meta_train_df,
            y_train=self.y_train,
            X_val=self.meta_test_df,
            y_val=self.y_test,
            params=params,
        )

        clf.fit(X=self.meta_train_df, y=self.y_train, **fit_params)
        self.level_two_model = clf
    
        # Get predictions
        test_preds = clf.predict_proba(self.meta_test_df)[:, 1]
        self.meta_test_df["target"] = test_preds >= 0.5

        return self.meta_test_df

    def predict(self, X: pd.DataFrame) -> pd.Series:
        """Make predictions on new data using the trained ensemble.

        Args:
            X: DataFrame with features to predict on

        Returns:
            Series with binary predictions
        """
        if not self.level_one_models or not self.level_two_model:
            raise RuntimeError(
                "Models have not been trained. Call fit_level_one_models and fit_level_two_model first."
            )

        # Get level 1 predictions
        meta_features = self.get_meta_features(X)
        log.debug(f"Meta features: {meta_features.head()}")
        # Get level 2 predictions
        predictions = self.level_two_model.predict(meta_features)

        return pd.Series(predictions, index=X.index)

    def get_meta_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """Get meta features for new data using the trained ensemble.

        Args:
            X: DataFrame with features to predict on

        Returns:
            DataFrame with meta features
        """
        if not self.level_one_models:
            raise RuntimeError(
                "Level 1 models have not been trained. Call fit_level_one_models first."
            )

        meta_features = pd.DataFrame(index=X.index)
        for model_name, model in self.level_one_models.items():
            meta_features[f"{model_name}_preds"] = model.predict_proba(X)[:, 1]

        return meta_features

    def predict_proba(self, X: pd.DataFrame) -> np.ndarray:
        """Make probability predictions on new data using the trained ensemble.

        Args:
            X: DataFrame with features to predict on

        Returns:
            Array of probability predictions
        """
        if not self.level_one_models or not self.level_two_model:
            raise RuntimeError(
                "Models have not been trained. Call fit_level_one_models and fit_level_two_model first."
            )

        # Get level 1 predictions
        meta_features = self.get_meta_features(X)

        # Get level 2 predictions
        predictions = self.level_two_model.predict_proba(meta_features)

        return predictions

    def get_best_params(self, model: Optional[str] = None):
        """
        Retrieve best parameters from study for a specific model or all models.

        Args:
                model: Optional model name. If None, returns parameters for all models.

        Returns:
                Dictionary of best parameters from study for requested model(s)
        """
        if model is not None:
            # Load existing study for specific model
            study = optuna.load_study(
                storage=Config.OPTUNA_STORAGE,
                study_name=self.level_one_study_names[model],
            )
            return study.best_params

        # Load studies for all models
        best_params = {}
        for model_, study_name in self.level_one_study_names.items():
            study = optuna.load_study(
                storage=Config.OPTUNA_STORAGE, study_name=study_name
            )
            best_params[model_] = study.best_params
        study = optuna.load_study(
            storage=Config.OPTUNA_STORAGE, study_name=self.level_two_study_name
        )
        best_params["lr_l2"] = study.best_params

        return best_params

    def print_best_params(self):
        """Print best parameters for all models in a formatted way."""
        for model, params in self.best_params.items():
            if model == "level2_lr":
                log.info("\nLevel 2 (Logistic Regression) Best Parameters:")
            else:
                log.info(
                    f"\nLevel 1 - {Config.MODELS.get(model, model)} Best Parameters:"
                )
            for param, value in params.items():
                log.info(f"\t{param}: {value}")

In [18]:
out10 = get_node_output("handle_outliers_node")

Getting outputs for node: handle_outliers_node
Node outputs: ['train_df_winsorized', 'test_df_winsorized']

Processing output: train_df_winsorized
Loading from filepath: /Users/krzysztofwojdalski/github_projects/qrt-stock-returns/data/05_model_input/train_df_winsorized.pkl
Loading Pickle file...
Successfully loaded dataframe with shape: (416339, 121)

Processing output: test_df_winsorized
Loading from filepath: /Users/krzysztofwojdalski/github_projects/qrt-stock-returns/data/05_model_input/test_df_winsorized.pkl
Loading Pickle file...
Successfully loaded dataframe with shape: (198429, 120)

Finished loading 2 outputs


## Split Data into Training and Test Sets

In [19]:
out11 = run_pipeline_node(
    "split_data_node",
    {
        "train_df_winsorized": out10["train_df_winsorized"],
        "params:model_options": conf_params["model_options"],
    },
)

[02/09/25 18:06:50] INFO     Running node: split_data_node:                                             ]8;id=234053;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=146316;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             split_data([train_df_winsorized;params:model_options]) ->                             
                             [X_train;X_test;y_train;y_test]                                                       

In [20]:
X_train = out11["X_train"]
X_test = out11["X_test"]
y_train = out11["y_train"]
y_test = out11["y_test"]

In [21]:
# Replace NaN values with median for each column
log.info(
    f"Removed NaN rows. New shapes - X_train: {X_train.shape}, X_test: {X_test.shape}"
)

ensemble = Ensemble(X_train, X_test, y_train, y_test)
ensemble.models
ensemble.get_best_params()

[02/09/25 18:06:51] INFO     Removed NaN rows. New shapes - X_train: (312254, 120), X_test:         ]8;id=91161;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3969982664.py\3969982664.py]8;;\:]8;id=619176;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3969982664.py#2\2]8;;\
                             (104085, 120)                                                                         


{
    'xgb': {
        'max_depth': 8,
        'n_estimators': 296,
        'learning_rate': 0.05665725247980031,
        'min_child_weight': 1,
        'gamma': 0.17931445252110315,
        'subsample': 0.8901629177609262,
        'colsample_bytree': 0.9834024429083399,
        'max_delta_step': 0,
        'reg_alpha': 0.4969962068742358,
        'reg_lambda': 0.04757581134478107,
        'colsample_bylevel': 0.753058634349968,
        'early_stopping_rounds': 15
    },
    'lgb': {
        'num_leaves': 96,
        'max_depth': 65,
        'learning_rate': 0.07078572504914454,
        'n_estimators': 478,
        'reg_alpha': 3.264340419698323,
        'reg_lambda': 0.8256835777730215,
        'min_child_samples': 36,
        'subsample_for_bin': 2791,
        'subsample': 0.6698139082108787,
        'colsample_bytree': 0.5231180629361427,
        'min_split_gain': 0.7437846596165149
    },
    'lr_l2': {'tol': 8.925642518980422e-05, 'C': 0.5102635371744328, 'solver': 'sag'}
}

In [ ]:
ensemble.fit_level_one_models(search=False)
ensemble.meta_train_df.head()
ensemble.meta_test_df.head()

# Get predictions for test data using the trained ensemble
# Get the level 2 model's coefficients to see how it weights the level 1 models



[02/09/25 18:18:15] INFO     Training level 1 models...                                            ]8;id=149811;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py\3708709130.py]8;;\:]8;id=277746;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py#49\49]8;;\

                    INFO     XGBoost:                                                              ]8;id=565158;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py\3708709130.py]8;;\:]8;id=275504;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py#52\52]8;;\

[02/09/25 18:18:16] INFO             Using existing best parameters...                             ]8;id=611878;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py\3708709130.py]8;;\:]8;id=418801;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py#70\70]8;;\
                                                                                                                   

                    INFO             Training model with optimal parameters...                     ]8;id=517488;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py\3708709130.py]8;;\:]8;id=95325;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py#72\72]8;;\
                                                                                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 ensemble.fit_level_one_models(search=False)                                                 │
│    2 ensemble.meta_train_df.head()                                                               │
│    3 ensemble.meta_test_df.head()                                                                │
│    4 test_predictions = ensemble.fit_level_two_model(search=False)                               │
│                                                                                                  │
│ /var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py:92 in             │
│ fit_level_one_models                                                                             │
│                                                                                                  │
│    89 │   │   │   │   params=params,                                                             │
│    90 │   │   │   )                                                                              │
│    91 │   │   │                                                                                  │
│ ❱  92 │   │   │   clf.fit(X=self.X_train, y=self.y_train, **fit_params)                          │
│    93 │   │   │   self.level_one_models[model] = clf                                             │
│    94 │   │   │                                                                                  │
│    95 │   │   │   # Get predictions for meta features                                            │
│                                                                                                  │
│ /Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/xgboost/core.py:72 │
│ 6 in inner_f                                                                                     │
│                                                                                                  │
│    723 │   │   │   │   warnings.warn(msg, FutureWarning)                                         │
│    724 │   │   │   for k, arg in zip(sig.parameters, args):                                      │
│    725 │   │   │   │   kwargs[k] = arg                                                           │
│ ❱  726 │   │   │   return func(**kwargs)                                                         │
│    727 │   │                                                                                     │
│    728 │   │   return inner_f                                                                    │
│    729                                                                                           │
│                                                                                                  │
│ /Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/xgboost/sklearn.py │
│ :1531 in fit                                                                                     │
│                                                                                                  │
│   1528 │   │   │   │   feature_types=self.feature_types,                                         │
│   1529 │   │   │   )                                                                             │
│   1530 │   │   │                                                                                 │
│ ❱ 1531 │   │   │   self._Booster = train(                                                        │
│   1532 │   │   │   │   params,                                                                   │
│   1533 │   │   │   │   train_dmatrix,                                                            │
│   1534 │   │   │   │   self.get_num_boosting_rounds(),     

In [33]:
test_predictions = ensemble.fit_level_two_model(search=False)

[02/09/25 18:29:34] INFO     Training a Logistic Regression model as level 2 model...             ]8;id=554816;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py\3708709130.py]8;;\:]8;id=263626;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py#108\108]8;;\

[02/09/25 18:29:38] INFO             Using existing best parameters...                            ]8;id=713328;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py\3708709130.py]8;;\:]8;id=755731;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py#127\127]8;;\
                                                                                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/491339588.py:1 in <module>      │
│                                                                                                  │
│ ❱ 1 test_predictions = ensemble.fit_level_two_model(search=False)                                │
│   2                                                                                              │
│                                                                                                  │
│ /var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/3708709130.py:150 in            │
│ fit_level_two_model                                                                              │
│                                                                                                  │
│   147 │   │   self.level_two_model = clf                                                         │
│   148 │   │                                                                                      │
│   149 │   │   # Get predictions                                                                  │
│ ❱ 150 │   │   test_preds = clf.predict_proba(self.meta_test_df)[:, 1]                            │
│   151 │   │   self.meta_test_df["target"] = test_preds >= 0.5                                    │
│   152 │   │                                                                                      │
│   153 │   │   return self.meta_test_df                                                           │
│                                                                                                  │
│ /Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/sklearn/linear_mod │
│ el/_logistic.py:1431 in predict_proba                                                            │
│                                                                                                  │
│   1428 │   │   │   )                                                                             │
│   1429 │   │   )                                                                                 │
│   1430 │   │   if ovr:                                                                           │
│ ❱ 1431 │   │   │   return super()._predict_proba_lr(X)                                           │
│   1432 │   │   else:                                                                             │
│   1433 │   │   │   decision = self.decision_function(X)                                          │
│   1434 │   │   │   if decision.ndim == 1:                                                        │
│                                                                                                  │
│ /Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/sklearn/linear_mod │
│ el/_base.py:397 in _predict_proba_lr                                                             │
│                                                                                                  │
│   394 │   │   1. / (1. + np.exp(-self.decision_function(X)));                                    │
│   395 │   │   multiclass is handled by normalizing that over all classes.                        │
│   396 │   │   """                                                                                │
│ ❱ 397 │   │   prob = self.decision_function(X)                                                   │
│   398 │   │   expit(prob, out=prob)                                                              │
│   399 │   │   if prob.ndim == 1:                                                                 │
│   400 │   │   │   return np.vstack([1 - prob, prob]).T                                           │
│                                                                                                  │
│ /Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python

In [26]:
test_predictions = ensemble.predict(out10["test_df_winsorized"])
test_predictions.to_csv("data/07_model_output/submission6.csv")

log.info("\nTest Set Predictions (first 5 rows):")
log.info(test_predictions[:5])


# Count disagreements between XGB and LGB predictions
# Get predictions for first 5 rows of test data

# Get level 1 predictions for each model

[02/09/25 18:16:34] INFO                                                                            ]8;id=523481;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/1489752414.py\1489752414.py]8;;\:]8;id=414850;file:///var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/1489752414.py#4\4]8;;\
                             Test Set Predictions (first 5 rows):                                                  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pandas/core/indexe │
│ s/base.py:3805 in get_loc                                                                        │
│                                                                                                  │
│   3802 │   │   """                                                                               │
│   3803 │   │   casted_key = self._maybe_cast_indexer(key)                                        │
│   3804 │   │   try:                                                                              │
│ ❱ 3805 │   │   │   return self._engine.get_loc(casted_key)                                       │
│   3806 │   │   except KeyError as err:                                                           │
│   3807 │   │   │   if isinstance(casted_key, slice) or (                                         │
│   3808 │   │   │   │   isinstance(casted_key, abc.Iterable)                                      │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:167                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:196                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.Float64HashTable.get_item:1698                                         │
│                                                                                                  │
│ in pandas._libs.hashtable.Float64HashTable.get_item:1722                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 5.0

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/_4/lvfgz8ld58b3qwkrcr3n3l_h0000gn/T/ipykernel_14823/1489752414.py:5 in <module>     │
│                                                                                                  │
│    2 test_predictions.to_csv("data/07_model_output/submission6.csv")                             │
│    3                                                                                             │
│    4 log.info("\nTest Set Predictions (first 5 rows):")                                          │
│ ❱  5 log.info(test_predictions[:5])                                                              │
│    6                                                                                             │
│    7                                                                                             │
│    8 # Count disagreements between XGB and LGB predictions                                       │
│                                                                                                  │
│ /Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pandas/core/series │
│ .py:1146 in __getitem__                                                                          │
│                                                                                                  │
│   1143 │   │                                                                                     │
│   1144 │   │   if isinstance(key, slice):                                                        │
│   1145 │   │   │   # Do slice check before somewhat-costly is_bool_indexer                       │
│ ❱ 1146 │   │   │   return self._getitem_slice(key)                                               │
│   1147 │   │                                                               

In [31]:

level2_weights = ensemble.get_best_params()
# log.info("\nLevel 2 Model Weights:")
# for model_name, weight in zip(['XGBoost', 'LightGBM'], level2_weights):
#     log.info(f"{model_name}: {weight:.4f}")
level2_weights["lr_l2"]


{'tol': 8.925642518980422e-05, 'C': 0.5102635371744328, 'solver': 'sag'}